In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
def get_newton_cotes(some_func, a_point, b_point, subsegment_functor, N):
    if (N - 1) % (subsegment_functor.get_nodes_number() - 1) != 0:
        old_n = N
        N += (subsegment_functor.get_nodes_number() -
              (N % (subsegment_functor.get_nodes_number() - 1)))
        print 'Newton cotes algorithm N=', old_n, ' has increased to ', N
    num_divs = (N - 1) / (subsegment_functor.get_nodes_number() - 1) + 1
    div_points = np.linspace(a_point, b_point, num_divs).astype(float)
    shifted_lists = zip(div_points[ :-1], div_points[1: ])
    subsegm_sums = map(lambda x: subsegment_functor(some_func, x[0], x[1]), shifted_lists)
    return sum(subsegm_sums)

def get_richardson_extrapolation_integral(some_func, a_point, b_point, subsegment_functor, N):
    cur_p = subsegment_functor.get_approximation_order()
    cur_divisor = float((1 << cur_p) -1)
    return get_newton_cotes(some_func, a_point, b_point, subsegment_functor,
                             2*N-1) + (get_newton_cotes(some_func, a_point, b_point, subsegment_functor,
                                 2*N-1) - get_newton_cotes(some_func, a_point, b_point, subsegment_functor,
                                 N))/cur_divisor

In [3]:
def get_gauss(some_func, a_point, b_point, gaus_N, legandre_roots_on_minus_one_one, 
             c_values_on_minus_one_one):
    get_real_legandre_points = (b_point - a_point) * 0.5 * np.array(legandre_roots_on_minus_one_one)
    get_real_legandre_points = get_real_legandre_points + ((a_point + b_point) * 0.5)
    c_real_values = (b_point - a_point) * 0.5 * np.array(c_values_on_minus_one_one)
    f_at_legandre_points = np.array(map(some_func, get_real_legandre_points))
    return np.sum(f_at_legandre_points * c_real_values)


In [4]:
def make_report_common(some_func, a_point, b_point, subsegment_functor, N,
                       gaus_N, legandre_roots_on_minus_one_one,
                       c_values_on_minus_one_one,
                       antiderivative_function = None):
    col_names = ['Newton-Cotes_' + str(N), 'Newton-Cotes_' + str(2*N-1),
                'Richardson_extrapolation', 'Gauss_' + str(gaus_N) + '_formula']
    if antiderivative_function is not None:
        col_names += ['Analytical answer']
    newt_cot_N_val = get_newton_cotes(some_func, a_point, b_point,
                                      subsegment_functor, N)
    newt_cot_2N_1_val = get_newton_cotes(some_func, a_point, b_point,
                                         subsegment_functor, 2*N-1)
    rich_approx_val = get_richardson_extrapolation_integral(some_func, a_point, b_point,
                                                            subsegment_functor, N)
    gaus_val = get_gauss(some_func, a_point, b_point, gaus_N,
                         legandre_roots_on_minus_one_one, 
                         c_values_on_minus_one_one)
    provided_values = [newt_cot_N_val, newt_cot_2N_1_val,
                       rich_approx_val, gaus_val]
    if antiderivative_function is not None:
        analytical_answer = antiderivative_function(b_point) - antiderivative_function(a_point)
        provided_values.append(analytical_answer)
    provided_values = np.array(provided_values).reshape((1, len(provided_values)))
    return pd.DataFrame(provided_values, columns = col_names)

In [5]:
def given_function(x_point):
    return 1./(4 * x_point * x_point + 4 * x_point + 5)

def antiderivative_function_given(x_point):
        return 0.25 * np.arctan(x_point + 0.5)

class simpson_functor:
    
    def __init__(self):
        return None
    
    def __call__(self, some_function, left_point, right_point):
        return (some_function(left_point) + 4. * some_function(0.5 * left_point + 0.5 * right_point) +
               some_function(right_point)) * (right_point - left_point)/6.
    
    def get_approximation_order(self):
        return 4
    
    def get_nodes_number(self):
        return 3
    
gaus_2_node_values = [np.sqrt(3)/3., -np.sqrt(3)/3.]
gaus_2_c_values = [1., 1.]

In [6]:
def make_report_to_show(N):
    a_point = 0
    b_point = 1
    table_to_draw = make_report_common(given_function, a_point, b_point,
                                      simpson_functor(), N, 2,
                                      gaus_2_node_values,
                                      gaus_2_c_values,
                                      antiderivative_function_given)
    print table_to_draw.to_string(index = False)
    real_ans = 0.25 * np.arctan(4. / 7.)
    print 'Correct answer from the book is ', real_ans

make_report_to_show(41)

In [8]:
"{0:.2f}".format(f)

NameError: name 'f' is not defined